In [5]:
import json
import sqlite3

conn = sqlite3.connect('rosterdb.sqlite')
cur = conn.cursor() #file handling

# Do some setup - create tables
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER PRIMARY KEY,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER PRIMARY KEY,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id, role)
)
''')

fname = input('Enter file name: ') # roster_data.json
if len(fname) < 1:
    fname = 'roster_data.json'

#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

#open data
str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0]
    title = entry[1]
    role = entry[2]

    # print((name, title))

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        ( user_id, course_id, role) )

    conn.commit()
    
    #data retrieve
sqlstr = '''SELECT User.name,Course.title, Member.role FROM User 
            JOIN Member JOIN Course 
            ON User.id = Member.user_id AND Member.course_id = Course.id
            ORDER BY User.name DESC, Course.title DESC, Member.role DESC LIMIT 2;
            '''

sqlstr2 =''' SELECT 'XYZZY' || hex(User.name || Course.title || Member.role ) AS X FROM User 
             JOIN Member JOIN Course 
             ON User.id = Member.user_id AND Member.course_id = Course.id
             ORDER BY X LIMIT 1;
        '''
    
    # Loop through the rows returned by the SQL query and print each album and its artist
for row in cur.execute(sqlstr):
    print(f"A: | {row[0]}, | {row[1]} | {row[2]}")
 
    #query data 
for row in cur.execute(sqlstr2):
    print(f"A: | {row[0]}")
    
# Close the database connection after the script finishes
conn.close()


Enter file name:  roster_data.json


A: | Zunera, | si206 | 0
A: | Zosia, | si110 | 0
A: | XYZZY41616D69726168736932303630
